In [26]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [28]:
tp_cols = ["siape", "formacao", "tipo_jornada_trabalho",
           "vinculo", "lotacao", "admissao"]

tp_df = pd.read_csv("./perfis/docentes.csv", sep=";")
tp_df = tp_df[tp_cols]

tp_df.head()

,siape,formacao,tipo_jornada_trabalho,vinculo,lotacao,admissao
0,1543339,MESTRADO,Dedicação exclusiva,Ativo Permanente,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA,2006/07/24 00:00:00.000000000
1,1554468,DOUTORADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,2008/09/12 00:00:00.000000000
2,1177821,MESTRADO,Dedicação exclusiva,Ativo Permanente,ESCOLA DE MÚSICA,1998/04/28 00:00:00.000000000
3,2360824,MESTRADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,2017/01/25 00:00:00.000000000
4,2364334,DOUTORADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,2009/10/13 00:00:00.000000000


In [29]:
tp_df.tail()

,siape,formacao,tipo_jornada_trabalho,vinculo,lotacao,admissao
2765,4246363,DOUTORADO,Dedicação exclusiva,Professor Visitante,INSTITUTO DE POLÍTICAS PÚBLICAS,2023/05/23 00:00:00.000000000
2766,3304576,DOUTORADO,Dedicação exclusiva,Professor Visitante,ESCOLA DE CIÊNCIAS E TECNOLOGIA,2022/08/12 00:00:00.000000000
2767,1056188,DOUTORADO,Dedicação exclusiva,Professor Visitante,DEPARTAMENTO DE BIOLOGIA CELULAR E GENÉTICA,2022/10/03 00:00:00.000000000
2768,3330361,DOUTORADO,Dedicação exclusiva,Professor Visitante,INSTITUTO METROPOLE DIGITAL,2023/02/15 00:00:00.000000000
2769,3309092,DOUTORADO,Dedicação exclusiva,Professor Visitante,DEPARTAMENTO DE GEOFÍSICA,2022/09/06 00:00:00.000000000


In [30]:
tp_df.dtypes

siape                     int64
formacao                 object
tipo_jornada_trabalho    object
vinculo                  object
lotacao                  object
admissao                 object
dtype: object

In [31]:
tp_df.describe()

,siape
count,2.770000e+03
mean,2.114588e+06
std,1.142222e+06
min,1.274600e+04
25%,1.297595e+06
50%,1.810985e+06
75%,2.722937e+06
max,9.350807e+06


In [32]:
qualis = ["siape", "revista_a1", "revista_a2", "revista_b1",
          "revista_b2", "revista_b3", "revista_b4", "revista_b5", "revista_c"]

ti_df_list = []
for year in range(2010, 2021):
    ti_df_y = pd.read_csv(
        "./indicadores/indicadores-pesquisa-" + str(year) + ".csv", sep=";")
    ti_df_y = ti_df_y[qualis]
    ti_df_list.append(ti_df_y)

ti_df = pd.concat(ti_df_list)
ti_df = ti_df.groupby("siape", as_index=False).sum()
ti_df.describe()

,siape,revista_a1,revista_a2,revista_b1,revista_b2,revista_b3,revista_b4,revista_b5,revista_c
count,2.756000e+03,2756.000000,2756.000000,2756.000000,2756.000000,2756.000000,2756.000000,2756.000000,2756.000000
mean,2.112227e+06,2.634978,1.933599,1.750000,1.269231,0.818215,0.568578,0.054790,0.889332
std,1.137681e+06,6.073799,3.917333,5.375914,2.891393,2.413658,1.778480,0.330389,3.855806
min,1.274600e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.296285e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.808676e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.721404e+06,3.000000,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,9.350807e+06,71.000000,51.000000,99.000000,46.000000,41.000000,32.000000,8.000000,124.000000


In [33]:
def num_semestres(data: str, data_atual: str):
    data = data[:10]
    anos = int(data_atual[6:]) - int(data[:4])
    if int(data_atual[3:5]) < 7 and int(data[5:7]) < 7 or int(data_atual[3:5]) >= 7 and int(data[5:7]) >= 7:
        return 2*anos
    elif int(data_atual[3:5]) < 7 and int(data[5:7]) > 7:
        return 2*anos - 1
    else:
        return 2*anos + 1


df = tp_df.merge(ti_df, on="siape", how="inner")
del df["siape"]

data_atual = "18/10/2023"

df['num_semestres'] = df['admissao'].apply(
    lambda x: num_semestres(x, data_atual))
df = df[["formacao", "tipo_jornada_trabalho", "vinculo", "lotacao", "num_semestres", "revista_a1",
         "revista_a2", "revista_b1", "revista_b2", "revista_b3", "revista_b4", "revista_c"]]

df.head()

,formacao,tipo_jornada_trabalho,vinculo,lotacao,num_semestres,revista_a1,revista_a2,revista_b1,revista_b2,revista_b3,revista_b4,revista_c
0,MESTRADO,Dedicação exclusiva,Ativo Permanente,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA,34,0,0,0,0,0,0,0
1,DOUTORADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,30,1,1,5,0,0,4,0
2,MESTRADO,Dedicação exclusiva,Ativo Permanente,ESCOLA DE MÚSICA,51,0,0,0,0,0,0,0
3,MESTRADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,13,0,0,0,0,0,0,0
4,DOUTORADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,28,0,0,0,1,0,0,0


In [34]:
df.dtypes

formacao                 object
tipo_jornada_trabalho    object
vinculo                  object
lotacao                  object
num_semestres             int64
revista_a1                int64
revista_a2                int64
revista_b1                int64
revista_b2                int64
revista_b3                int64
revista_b4                int64
revista_c                 int64
dtype: object

In [35]:
print("existem", len(df), "entradas diferentes;")
print("existem", len(df["formacao"].unique()), "formacoes diferentes;")
print("existem", len(df["tipo_jornada_trabalho"].unique()),
      "tipos de jornada de trabalho diferentes;")
print("existem", len(df["vinculo"].unique()), "tipos de vinculo diferentes;")
print("existem", len(df["lotacao"].unique()), "lotacoes diferentes;")
print("existem", len(df["num_semestres"].unique()),
      "datas de admissao diferentes;")

existem 2753 entradas diferentes;
existem 6 formacoes diferentes;
existem 3 tipos de jornada de trabalho diferentes;
existem 8 tipos de vinculo diferentes;
existem 135 lotacoes diferentes;
existem 89 datas de admissao diferentes;


In [36]:
for formacao in df["formacao"].unique():
    print(formacao, ": ", len(df[df["formacao"] == formacao]), sep="")

MESTRADO: 431
DOUTORADO: 2189
ESPECIALIZAÇÃO: 120
GRADUAÇÃO: 11
PÓS-DOUTORADO: 1
DESCONHECIDA: 1


In [37]:
df = df[df["formacao"] != "DESCONHECIDA"]
for formacao in df["formacao"].unique():
    print(formacao, ": ", len(df[df["formacao"] == formacao]), sep="")

MESTRADO: 431
DOUTORADO: 2189
ESPECIALIZAÇÃO: 120
GRADUAÇÃO: 11
PÓS-DOUTORADO: 1


In [38]:
for tipo_jornada_trabalho in df["tipo_jornada_trabalho"].unique():
    print(tipo_jornada_trabalho, ": ", len(
        df[df["tipo_jornada_trabalho"] == tipo_jornada_trabalho]), sep="")

Dedicação exclusiva           : 2155
20 horas semanais             : 307
40 horas semanais             : 290


In [39]:
for vinculo in df["vinculo"].unique():
    print(vinculo, ": ", len(df[df["vinculo"] == vinculo]), sep="")

Ativo Permanente: 2376
Celetista: 1
Colaborador PCCTAE e Magistério Federal: 2
Excedente de lotação: 3
Exercicio provisorio: 18
Professor Substituto: 260
Professor Temporario: 50
Professor Visitante: 42


In [40]:
df = df[df["vinculo"] != "Celetista"]
df = df[df["vinculo"] != "Colaborador PCCTAE e Magistério Federal"]
df = df[df["vinculo"] != "Excedente de lotação"]
for vinculo in df["vinculo"].unique():
    print(vinculo, ": ", len(df[df["vinculo"] == vinculo]), sep="")

Ativo Permanente: 2376
Exercicio provisorio: 18
Professor Substituto: 260
Professor Temporario: 50
Professor Visitante: 42


In [41]:
df.describe()

,num_semestres,revista_a1,revista_a2,revista_b1,revista_b2,revista_b3,revista_b4,revista_c
count,2746.000000,2746.000000,2746.000000,2746.000000,2746.000000,2746.000000,2746.000000,2746.000000
mean,25.367444,2.634377,1.932265,1.750546,1.268026,0.811726,0.567371,0.888201
std,20.846636,6.079585,3.920029,5.382835,2.893793,2.387709,1.775777,3.857700
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,30.000000,3.000000,2.000000,2.000000,1.000000,1.000000,0.000000,1.000000
max,97.000000,71.000000,51.000000,99.000000,46.000000,41.000000,32.000000,124.000000


In [42]:
df.describe(include="object")

,formacao,tipo_jornada_trabalho,vinculo,lotacao
count,2746,2746,2746,2746
unique,5,3,5,134
top,DOUTORADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ
freq,2183,2149,2376,120


In [43]:
keep = ['revista_a1',
        'revista_a2',
        'revista_b1',
        'revista_b2',
        'revista_b3',
        'revista_b4',
        'revista_c']

X = df.drop(keep, axis=1).copy()
X

,formacao,tipo_jornada_trabalho,vinculo,lotacao,num_semestres
0,MESTRADO,Dedicação exclusiva,Ativo Permanente,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA,34
1,DOUTORADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,30
2,MESTRADO,Dedicação exclusiva,Ativo Permanente,ESCOLA DE MÚSICA,51
3,MESTRADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,13
4,DOUTORADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,28
...,...,...,...,...,...
2748,DOUTORADO,Dedicação exclusiva,Professor Visitante,INSTITUTO DE POLÍTICAS PÚBLICAS,1
2749,DOUTORADO,Dedicação exclusiva,Professor Visitante,ESCOLA DE CIÊNCIAS E TECNOLOGIA,2
2750,DOUTORADO,Dedicação exclusiva,Professor Visitante,DEPARTAMENTO DE BIOLOGIA CELULAR E GENÉTICA,2
2751,DOUTORADO,Dedicação exclusiva,Professor Visitante,INSTITUTO METROPOLE DIGITAL,1


In [44]:
[ya1,
 ya2,
 yb1,
 yb2,
 yb3,
 yb4,
 yc] = [df[col].copy() for col in keep]

[ya1, ya2, yb1, yb2, yb3, yb4, yc]

[0        0
 1        1
 2        0
 3        0
 4        0
         ..
 2748     0
 2749     8
 2750     4
 2751    44
 2752     4
 Name: revista_a1, Length: 2746, dtype: int64,
 0       0
 1       1
 2       0
 3       0
 4       0
        ..
 2748    0
 2749    4
 2750    2
 2751    1
 2752    0
 Name: revista_a2, Length: 2746, dtype: int64,
 0       0
 1       5
 2       0
 3       0
 4       0
        ..
 2748    0
 2749    7
 2750    5
 2751    0
 2752    0
 Name: revista_b1, Length: 2746, dtype: int64,
 0       0
 1       0
 2       0
 3       0
 4       1
        ..
 2748    1
 2749    0
 2750    0
 2751    0
 2752    0
 Name: revista_b2, Length: 2746, dtype: int64,
 0       0
 1       0
 2       0
 3       0
 4       0
        ..
 2748    0
 2749    2
 2750    0
 2751    0
 2752    0
 Name: revista_b3, Length: 2746, dtype: int64,
 0       0
 1       4
 2       0
 3       0
 4       0
        ..
 2748    0
 2749    0
 2750    0
 2751    0
 2752    0
 Name: revista_b4, Length: 2

In [45]:
X_encoded = pd.get_dummies(X, columns=['formacao',
                                       'tipo_jornada_trabalho',
                                       'vinculo',
                                       'lotacao',
                                       'num_semestres'])

X_encoded

,formacao_DOUTORADO,formacao_ESPECIALIZAÇÃO,formacao_GRADUAÇÃO,formacao_MESTRADO,formacao_PÓS-DOUTORADO,tipo_jornada_trabalho_20 horas semanais,tipo_jornada_trabalho_40 horas semanais,tipo_jornada_trabalho_Dedicação exclusiva,vinculo_Ativo Permanente,vinculo_Exercicio provisorio,...,num_semestres_88,num_semestres_89,num_semestres_90,num_semestres_91,num_semestres_92,num_semestres_93,num_semestres_94,num_semestres_95,num_semestres_96,num_semestres_97
0,False,False,False,True,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2748,True,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2749,True,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2750,True,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2751,True,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [46]:
[ya1.unique(),
 ya2.unique(),
 yb1.unique(),
 yb2.unique(),
 yb3.unique(),
 yb4.unique(),
 yc.unique()]

[array([ 0,  1,  3,  2,  4, 14,  8,  5,  6, 10, 13, 11, 17, 27, 16, 15,  7,
        19, 18, 31,  9, 26, 46, 32, 30, 12, 70, 44, 20, 37, 22, 25, 35, 41,
        56, 34, 60, 29, 21, 36, 23, 45, 51, 24, 71, 39, 49, 28, 38, 33, 40,
        43, 42], dtype=int64),
 array([ 0,  1,  5,  2,  3,  9,  4, 12,  6,  8, 14, 13, 11,  7, 16, 10, 15,
        19, 23, 27, 51, 20, 17, 24, 18, 25, 22, 33, 31, 21, 30, 29, 26, 35],
       dtype=int64),
 array([ 0,  5,  1,  3, 25,  4,  9,  2, 17,  6, 14, 21, 42,  7,  8, 36, 12,
        33, 10, 27, 50, 47, 32, 20, 16, 19, 18, 40, 13, 15, 11, 29, 30, 24,
        98, 75, 64, 23, 22, 41, 37, 99, 71, 68], dtype=int64),
 array([ 0,  1,  2,  3, 14,  4,  5,  6,  8, 17, 10,  9, 15, 22,  7, 13, 11,
        20, 12, 19, 16, 32, 18, 28, 46], dtype=int64),
 array([ 0,  1,  3,  2,  4, 16,  5,  9, 10, 12,  8,  7, 19, 36, 14, 15,  6,
        28, 29, 13, 20, 17, 30, 11, 41, 24], dtype=int64),
 array([ 0,  4,  1,  2,  3,  8,  6,  5, 24,  9,  7, 13, 16, 19, 11, 12, 14,
        15

In [47]:
[(sum(ya1) / len(ya1)),
 (sum(ya2) / len(ya2)),
 (sum(yb1) / len(yb1)),
 (sum(yb2) / len(yb2)),
 (sum(yb3) / len(yb3)),
 (sum(yb4) / len(yb4)),
 (sum(yc) / len(yc))]

[2.634377276037873,
 1.9322651128914785,
 1.750546249089585,
 1.2680262199563002,
 0.8117261471230881,
 0.5673707210487983,
 0.8882010196649672]

In [48]:
# Lista de variáveis alvo
target_vars = [(ya1, "ya1"), (ya2, "ya2"), (yb1, "yb1"), (yb2, "yb2"), (yb3, "yb3"), (yb4, "yb4"), (yc, "yc")]

# Dicionário para armazenar os conjuntos de treinamento e teste
train_test_sets = {}

# Loop sobre as variáveis alvo
for var, var_name in target_vars:
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, var, random_state=42)
    train_test_sets[var_name] = (X_train, X_test, y_train, y_test)

# Agora, você pode acessar os conjuntos de treinamento e teste usando o nome da variável alvo desejada
X_train_ya1, X_test_ya1, ya1_train, ya1_test = train_test_sets["ya1"]
X_train_ya2, X_test_ya2, ya2_train, ya2_test = train_test_sets["ya2"]
X_train_yb1, X_test_yb1, yb1_train, yb1_test = train_test_sets["yb1"]
X_train_yb2, X_test_yb2, yb2_train, yb2_test = train_test_sets["yb2"]
X_train_yb3, X_test_yb3, yb3_train, yb3_test = train_test_sets["yb3"]
X_train_yb4, X_test_yb4, yb4_train, yb4_test = train_test_sets["yb4"]
X_train_yc, X_test_yc, yc_train, yc_test = train_test_sets["yc"]


In [49]:
[(sum(ya1_train) / len(ya1_train)),
 (sum(ya2_train) / len(ya2_train)),
 (sum(yb1_train) / len(yb1_train)),
 (sum(yb2_train) / len(yb2_train)),
 (sum(yb3_train) / len(yb3_train)),
 (sum(yb4_train) / len(yb4_train)),
 (sum(yc_train) / len(yc_train))]

[2.5638659543467703,
 1.8989800874210783,
 1.6775133559980573,
 1.2423506556580866,
 0.8338999514327343,
 0.5900922778047596,
 0.9290917921321029]

In [50]:
[(sum(ya1_test) / len(ya1_test)),
 (sum(ya2_test) / len(ya2_test)),
 (sum(yb1_test) / len(yb1_test)),
 (sum(yb2_test) / len(yb2_test)),
 (sum(yb3_test) / len(yb3_test)),
 (sum(yb4_test) / len(yb4_test)),
 (sum(yc_test) / len(yc_test))]

[2.8457059679767105,
 2.032023289665211,
 1.9694323144104804,
 1.3449781659388647,
 0.745269286754003,
 0.4992721979621543,
 0.7656477438136827]

In [51]:
clf_xgb = xgb.XGBClassifier(object='binary:logistic', missing=None, seed=42)
clf_xgb.fit(X_train,
            ya1_train,
            verbose=True,
            early_stopping_rounds=10,
            eval_metric='aucpr',
            eval_set=[(X_test, ya1_test)])

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46], got [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 31 32 33 34 35 36 37 38 39 40 41 43 44 45 51 56 60]